In [1]:
# coding:utf8
import os
import ipdb
import torch as t
import torchvision as tv
import tqdm
from model import NetG, NetD
from torch.autograd import Variable
from torchnet.meter import AverageValueMeter
import visdom

In [13]:
class Config(object):
    data_path = './data/'  # 数据集存放路径
    num_workers = 1  # 多进程加载数据所用的进程数
    image_size = 96  # 图片尺寸
    batch_size = 128
    max_epoch = 31
    lr1 = 2e-4  # 生成器的学习率
    lr2 = 2e-4  # 判别器的学习率
    beta1 = 0.5  # Adam优化器的beta1参数
    gpu = False  # 是否使用GPU
    nz = 100  # 噪声维度
    ngf = 64  # 生成器feature map数
    ndf = 64  # 判别器feature map数

    save_path = './imgs/'  # 生成图片保存路径

    vis = True  # 是否使用visdom可视化
    env = 'GAN'  # visdom的env
    plot_every = 20  # 每间隔20 batch，visdom画图一次

    debug_file = './tmp/debuggan'  # 存在该文件则进入debug模式
    d_every = 1  # 每1个batch训练一次判别器
    g_every = 5  # 每5个batch训练一次生成器
    decay_every = 10  # 每10个epoch保存一次模型
#     netd_path = None  # 'checkpoints/netd_.pth' #预训练模型 加载参数
#     netg_path = None  # 'checkpoints/netg_211.pth'
    netd_path = './checkpoints/netd_29.pth'
    netg_path = './checkpoints/netg_29.pth'

    # 只测试不训练
    gen_img = './imgs/result.png'
    # 从512张生成的图片中保存最好的64张
    gen_num = 64
    gen_search_num = 512
    gen_mean = 0  # 噪声的均值
    gen_std = 1  # 噪声的方差


opt = Config()

## 随机输入512个100\*1\*1的噪声，用已经训练好的NetG得到分数高的64张3\*96\*96的图片

In [21]:
netg, netd = NetG(opt).eval(), NetD(opt).eval()
noises = t.randn(opt.gen_search_num, opt.nz, 1, 1).normal_(opt.gen_mean, opt.gen_std)
noises = Variable(noises, volatile=True)

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  app.launch_new_instance()


In [22]:
map_location = lambda storage, loc: storage
netd.load_state_dict(t.load(opt.netd_path, map_location=map_location))
netg.load_state_dict(t.load(opt.netg_path, map_location=map_location))

In [23]:
fake_img = netg(noises)
scores = netd(fake_img).data

In [24]:
# 挑选最好的某几张
indexs = scores.topk(opt.gen_num)[1]
result = []
for ii in indexs:
    result.append(fake_img.data[ii])

In [25]:
# 保存图片
tv.utils.save_image(t.stack(result), opt.gen_img, normalize=True, range=(-1, 1))